In [1]:
using Pkg
Pkg.activate(expanduser("./"))
Pkg.update()

  Activating project at `~/projects/TinyMachines.jl/examples`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `git@github.com:cirobr/TinyMachines.jl.git`
    Updating `~/projects/TinyMachines.jl/examples/Project.toml`
  [5d0fdf5f] ~ TinyMachines v0.1.5 `git@github.com:cirobr/TinyMachines.jl.git#v0.1.5rc3` ⇒ v0.1.5 `git@github.com:cirobr/TinyMachines.jl.git#v0.1.5rc3`
    Updating `~/projects/TinyMachines.jl/examples/Manifest.toml`
  [5d0fdf5f] ~ TinyMachines v0.1.5 `git@github.com:cirobr/TinyMachines.jl.git#v0.1.5rc3` ⇒ v0.1.5 `git@github.com:cirobr/TinyMachines.jl.git#v0.1.5rc3`
Precompiling project...
   4178.2 ms  ✓ TinyMachines
  1 dependency successfully precompiled in 9 seconds. 220 already precompiled.


In [2]:
using TinyMachines; tm=TinyMachines
using Flux
using CUDA
dev = CUDA.has_cuda_gpu() ? gpu : cpu

gpu (generic function with 2 methods)

In [3]:
# Synth data
Xs = rand(Float32, 64,64,3,10)
ys = rand(Bool, 64,64,2,10)

data = Flux.DataLoader((Xs, ys)) |> dev

10-element DataLoader(::MLUtils.MappedData{:auto, typeof(gpu), Tuple{Array{Float32, 4}, Array{Bool, 4}}})
  with first element:
  (64×64×3×1 CuArray{Float32, 4, CUDA.DeviceMemory}, 64×64×2×1 CuArray{Bool, 4, CUDA.DeviceMemory},)

In [4]:
x=randn(Float32, 3,3,1,1)

3×3×1×1 Array{Float32, 4}:
[:, :, 1, 1] =
  0.696602  0.388064  -0.638989
 -0.358273  0.24764   -0.168338
  0.410825  0.384498   1.27455

In [5]:
tm.PReLUlayer(1)(x)

3×3×1×1 Array{Float32, 4}:
[:, :, 1, 1] =
  0.696602  0.388064  -0.489822
 -0.274637  0.24764   -0.129041
  0.410825  0.384498   1.27455

In [6]:
# model
model = ESPnet(3,2) |> dev
model.encoder[1]

Chain(
  Conv((3, 3), 3 => 16, pad=1, stride=2),  # 448 parameters
  BatchNorm(16),                        # 32 parameters, plus 32
  TinyMachines.PReLUlayer(Float32[0.5993325, 0.3100606, 0.4675081, 0.23985612, 0.030973144, 0.6274185, 0.52000165, 0.47333127, 0.57612556, 0.9193262, 0.23416409, 0.23618303, 0.92490333, 0.23380512, 0.84719485, 0.2120533]),  # 16 parameters
)         # Total: 5 trainable arrays, 496 parameters,
          # plus 2 non-trainable, 32 parameters, summarysize 1.203 KiB.

In [7]:
# loss
loss(model,x,y) = Flux.crossentropy(model(x), y, dims=3)

loss (generic function with 1 method)

In [8]:
# optimizer
opt = Flux.Adam()
opt_state = Flux.setup(opt, model)

(downsampling = (), encoder = (layers = (e1 = (layers = ((σ = (), weight = Leaf(Adam(eta=0.001, beta=(0.9, 0.999), epsilon=1.0e-8), (Float32[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;;; … ;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0], Float32[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0

In [9]:
# train one epoch
Flux.train!(loss, model, data, opt_state)

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size: a has axes Base.OneTo(32) and b has axes Base.OneTo(16)

In [10]:
# inference
X = rand(Float32, 64,64,3,1) |> dev
ŷ = model(X) |> cpu
@assert size(ŷ) == (64,64,2,1)

DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size